In [8]:
# === IMPORTACIÓN DE LIBRERÍAS ===
import os                   
import requests             
import pandas as pd         
import json                 
from dotenv import load_dotenv  

In [9]:
load_dotenv()
URL = "https://cramer.buk.cl/api/v1/chile/employees"

In [10]:
# === AUTENTICACIÓN Y HEADERS ===
# Obtiene el token de autenticación desde variable de entorno (más seguro)
TOKEN = os.getenv("BUK_AUTH_TOKEN")
if not TOKEN:
    # Si no encuentra el token, termina el programa con mensaje de error
    raise SystemExit("ERROR: Define la variable de entorno BUK_AUTH_TOKEN con tu token")

print("✅ Token encontrado exitosamente")

# Configura los headers que se enviarán en cada petición HTTP
HEADERS = {
    "auth_token": TOKEN,           # Token de autenticación para la API
    "Accept": "application/json",  # Le dice al servidor que esperamos respuesta en JSON
    "Content-Type": "application/json"  # Especifica el tipo de contenido que enviamos
}

✅ Token encontrado exitosamente


In [11]:
# === FUNCIÓN PARA OBTENER TODOS LOS EMPLEADOS  ===
def fetch_all_employees():
    """
    Obtiene todos los empleados manejando paginación automáticamente.
    Retorna una lista con todos los registros de todas las páginas.
    """
    url = URL            
    all_records = []            # acumular todos los resultados
    page_count = 0              # Contador de páginas procesadas
    
    print("🔄 Iniciando descarga de empleados...")
    # Bucle que continúa mientras exista una URL para la siguiente página
    while url:
        try:
            print(f"📄 Procesando página {page_count + 1}...")
            
            # Realiza petición GET con timeout de 15 segundos
            response = requests.get(url, headers=HEADERS, timeout=15)
            
            # Lanza excepción si el status code no es exitoso (200-299)
            response.raise_for_status()
            
            # Convierte la respuesta JSON a diccionario de Python
            json_data = response.json()
            
            # Extrae los datos de empleados (estructura típica: {"data": [...], "pagination": {...}})
            employees_data = json_data.get("data", [])
            
            # Añade los empleados de esta página a la lista total
            all_records.extend(employees_data)
            
            print(f"   ✅ {len(employees_data)} empleados obtenidos de esta página")
            
            # Busca información de paginación para obtener la siguiente página
            pagination_info = json_data.get("pagination", {})
            url = pagination_info.get("next")  # URL de la siguiente página (None si no hay más)
            
            page_count += 1
            
        except requests.HTTPError as e:
            # Maneja errores HTTP específicos (400, 401, 404, 500, etc.)
            print(f"❌ Error HTTP: {e}")
            print(f"   Status Code: {getattr(e.response, 'status_code', 'N/A')}")
            print(f"   Respuesta del servidor: {getattr(e.response, 'text', 'Sin contenido')}")
            break
            
        except requests.RequestException as e:
            # Maneja otros errores de conexión (timeout, DNS, etc.)
            print(f"❌ Error de conexión: {e}")
            break
            
        except ValueError as e:
            # Maneja errores de JSON inválido
            print(f"❌ Error al procesar JSON: {e}")
            print(f"   Respuesta recibida: {response.text[:200]}...")
            break
    
    print(f"🎉 Descarga completada: {len(all_records)} empleados en {page_count} páginas")
    return all_records

In [ ]:
# === FUNCIÓN PARA LIMPIAR Y GUARDAR DATOS ===
def normalize_and_save_data(records, output_file="empleados_buk.csv"):
    """
    Normaliza los datos JSON y los guarda en un archivo CSV.
    Aplica limpieza básica de datos y conversiones de tipos.
    """
    # Verifica si hay datos para procesar
    if not records:
        print("⚠️  No hay registros para procesar")
        return None
    
    print(f"🔧 Procesando {len(records)} registros...")
    
    # Convierte JSON anidado a DataFrame plano
    # sep="_" hace que campos anidados se conviertan en columnas con guión bajo
    # Ejemplo: {"address": {"street": "Main"}} → "address_street": "Main"
    df = pd.json_normalize(records, sep="_")
    
    print(f"📊 DataFrame creado con {len(df)} filas y {len(df.columns)} columnas")
    
    # === LIMPIEZA DE DATOS ESPECÍFICA ===
    
    # Convierte campos numéricos (salarios, etc.)
    numeric_fields = ["base_wage", "salary", "wage", "amount"]
    for field in numeric_fields:
        if field in df.columns:
            # to_numeric convierte a número, errors='coerce' pone NaN si no puede convertir
            df[field] = pd.to_numeric(df[field], errors="coerce")
            print(f"   ✅ Campo '{field}' convertido a numérico")
    
    # Convierte campos de fecha automáticamente
    date_keywords = ["date", "fecha", "created_at", "updated_at", "active_since", "start_date", "end_date"]
    for column in df.columns:
        # Busca si el nombre de la columna contiene palabras clave de fecha
        if any(keyword in column.lower() for keyword in date_keywords):
            # Convierte a datetime, errors='coerce' pone NaT si no puede convertir
            df[column] = pd.to_datetime(df[column], errors="coerce")
            print(f"   ✅ Campo '{column}' convertido a fecha")
    
    # Guarda el DataFrame en archivo CSV
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"💾 Datos guardados en '{output_file}'")
    
    # Muestra estadísticas básicas del dataset
    print(f"\n📈 RESUMEN DEL DATASET:")
    print(f"   • Filas: {len(df)}")
    print(f"   • Columnas: {len(df.columns)}")
    print(f"   • Primeras 5 columnas: {list(df.columns[:5])}")
    
    return df

# === FUNCIÓN PARA MOSTRAR MUESTRA DE DATOS ===
def show_sample_data(records, sample_size=2):
    """
    Muestra una muestra de los datos en formato JSON legible.
    Útil para inspeccionar la estructura de los datos.
    """
    if not records:
        print("⚠️  No hay datos para mostrar")
        return
    
    print(f"\n🔍 MUESTRA DE DATOS (primeros {sample_size} registros):")
    print("=" * 50)
    
    # Toma solo los primeros registros para no saturar la pantalla
    sample = records[:sample_size]
    
    # json.dumps formatea el JSON de manera legible
    # indent=2: indentación de 2 espacios
    # ensure_ascii=False: permite caracteres especiales (tildes, ñ, etc.)
    formatted_json = json.dumps(sample, indent=2, ensure_ascii=False)
    print(formatted_json)
    
 

In [ ]:
# === FUNCIÓN PRINCIPAL QUE EJECUTA TODO EL PROCESO ===
def main():
    """
    Función principal que orquesta todo el proceso:
    1. Obtiene todos los empleados
    2. Muestra muestra de datos
    3. Procesa y guarda en CSV
    """
    print("🚀 INICIANDO PROCESO DE EXTRACCIÓN DE EMPLEADOS BUK")
    print("=" * 60)
    
    # Paso 1: Obtener todos los empleados con paginación
    empleados = fetch_all_employees()
    
    if not empleados:
        print("❌ No se pudieron obtener empleados. Revisa tu token y conexión.")
        return
    
    # Paso 2: Mostrar muestra de datos para inspección
    show_sample_data(empleados, sample_size=1)
    
    # Paso 3: Procesar y guardar datos
    df = normalize_and_save_data(empleados)
    
    if df is not None:
        print(f"\n🎉 PROCESO COMPLETADO EXITOSAMENTE")
        print(f"   • {len(empleados)} empleados descargados")
        print(f"   • Datos guardados en 'empleados_buk.csv'")
        print(f"   • DataFrame disponible para análisis adicional")
    
    return df

# === EJECUCIÓN DEL PROGRAMA ===
if __name__ == "__main__":
    # Solo ejecuta main() si el archivo se ejecuta directamente
    # (no si se importa como módulo)
    resultado = main()

🚀 INICIANDO PROCESO DE EXTRACCIÓN DE EMPLEADOS BUK
🔄 Iniciando descarga de empleados...
📄 Procesando página 1...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 2...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 3...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 4...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 5...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 6...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 7...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 8...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 9...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 10...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 11...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 12...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 13...
   ✅ 25 empleados obtenidos de esta página
📄 Procesando página 14..

C:\Users\gpavez\AppData\Local\Temp\ipykernel_10488\127549338.py:37: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[column] = pd.to_datetime(df[column], errors="coerce")


💾 Datos guardados en 'empleados_buk.csv'

📈 RESUMEN DEL DATASET:
   • Filas: 1094
   • Columnas: 139
   • Primeras 5 columnas: ['person_id', 'id', 'picture_url', 'first_name', 'surname']

🎉 PROCESO COMPLETADO EXITOSAMENTE
   • 1094 empleados descargados
   • Datos guardados en 'empleados_buk.csv'
   • DataFrame disponible para análisis adicional
